In [3]:
from IPython.display import display, HTML

display(
    HTML("""
<style>
* {font-family:D2Coding;}
div.container{width:87% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input { font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:3px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 1. 데이터 가져오기

In [40]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

df = pd.read_csv('/Volumes/DATA/mbc/ai_x/download/shareData/부동산_250213/최종전국평당분양가격(결측치제외).csv', encoding='cp949')
df.head()

,지역명,연도,월,평당분양가격
0,서울,2013,12,18189.0
1,부산,2013,12,8111.0
2,대구,2013,12,8080.0
3,인천,2013,12,10204.0
4,광주,2013,12,6098.0


# 2. 지역명의 라벨 인코딩

- 지역명을 라벨인코딩한 지역명
- 분석할 경우 원핫인코딩까지 할 것을 추천

In [41]:
le = LabelEncoder()
df['지역명2'] = le.fit_transform(df['지역명'])
df

,지역명,연도,월,평당분양가격,지역명2
0,서울,2013,12,18189.0,8
1,부산,2013,12,8111.0,7
2,대구,2013,12,8080.0,5
3,인천,2013,12,10204.0,11
4,광주,2013,12,6098.0,4
...,...,...,...,...,...
2171,전북,2024,8,12058.2,13
2172,전남,2024,8,13120.8,12
2173,경북,2024,8,13827.0,3
2174,경남,2024,8,13252.8,2


# 3. nomalization 스케일 조정
- 입력변수(지역명2, 연도, 월)와 타켓변수(평당분양가격) 따로 스케일 조정(Minmaxscaler 이용)
- 지역명2n, 연도n, 월n 필드 추가

In [42]:
minmax_scaler = MinMaxScaler()

df[['지역명2n', '연도n', '월n', '평당분양가격n']] = minmax_scaler.fit_transform(
    df[['지역명2', '연도', '월', '평당분양가격']]
)
df

,지역명,연도,월,평당분양가격,지역명2,지역명2n,연도n,월n,평당분양가격n
0,서울,2013,12,18189.0,8,0.5000,0.0,1.000000,0.328198
1,부산,2013,12,8111.0,7,0.4375,0.0,1.000000,0.065274
2,대구,2013,12,8080.0,5,0.3125,0.0,1.000000,0.064466
3,인천,2013,12,10204.0,11,0.6875,0.0,1.000000,0.119878
4,광주,2013,12,6098.0,4,0.2500,0.0,1.000000,0.012757
...,...,...,...,...,...,...,...,...,...
2171,전북,2024,8,12058.2,13,0.8125,1.0,0.636364,0.168252
2172,전남,2024,8,13120.8,12,0.7500,1.0,0.636364,0.195974
2173,경북,2024,8,13827.0,3,0.1875,1.0,0.636364,0.214398
2174,경남,2024,8,13252.8,2,0.1250,1.0,0.636364,0.199418


# 4. standarization 스케일 조정

- 입력변수(지역명2, 연도, 월)와 타켓변수(평당분양가격) 따로 스케일 조정(Standarization 이용)
- 지역명2s, 연도s, 월s 필드 추가

In [23]:
standard_scaler = StandardScaler()
df[['지역명2s', '연도s', '월s', '평당분양가격s']] = standard_scaler.fit_transform(
    df[['지역명2', '연도', '월', '평당분양가격']]
)
df

,지역명,연도,월,평당분양가격,지역명2,지역명2n,연도n,월n,평당분양가격n,지역명2s,연도s,월s,평당분양가격s
0,서울,2013,12,18189.0,8,0.5000,0.0,1.000000,0.328198,0.000000,-1.875367,1.62196,1.167561
1,부산,2013,12,8111.0,7,0.4375,0.0,1.000000,0.065274,-0.204124,-1.875367,1.62196,-0.727778
2,대구,2013,12,8080.0,5,0.3125,0.0,1.000000,0.064466,-0.612372,-1.875367,1.62196,-0.733608
3,인천,2013,12,10204.0,11,0.6875,0.0,1.000000,0.119878,0.612372,-1.875367,1.62196,-0.334154
4,광주,2013,12,6098.0,4,0.2500,0.0,1.000000,0.012757,-0.816497,-1.875367,1.62196,-1.106357
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2171,전북,2024,8,12058.2,13,0.8125,1.0,0.636364,0.168252,1.020621,1.664199,0.46374,0.014560
2172,전남,2024,8,13120.8,12,0.7500,1.0,0.636364,0.195974,0.816497,1.664199,0.46374,0.214400
2173,경북,2024,8,13827.0,3,0.1875,1.0,0.636364,0.214398,-1.020621,1.664199,0.46374,0.347213
2174,경남,2024,8,13252.8,2,0.1250,1.0,0.636364,0.199418,-1.224745,1.664199,0.46374,0.239225
